NOTE: I had the data locally so I just used it...

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_excel("AHD.xlsx")

In [3]:
df.head()

,Question,Answer,Category
0,اجريت صورة رنين مغناطيسي للثدي ظهر منطقة تعزيز...,المهم بالموضوع ما هي درجة ال BIRADS الموصوفة ل...,جراحة عامة
1,انا اعاني من اشياء تجبرني على فعل امور لا اريد...,الف سلامه عليك ، انت في حاجه الي معالج نفسي لم...,أمراض نفسية
2,أريد الاستفسار عن مضاعفات عملية استئصال ورم ال...,كغيرها من العمليات الجراحية ترتبط عملية استئصا...,أمراض الغدد الصماء
3,اليوم حاسس ان عندى حاجه فى الدبر من جوا مش عاي...,يغلب ان تعكس الحالة بواسير شرجية داخلية من الد...,جراحة عامة
4,ليه فجأة بحس انى مش عارفة اتنفس وجسمى بينمل وي...,من المحتمل عضوي او تاثير جانبي من اي ادويه بيت...,أمراض نفسية


In [4]:
len(df)

808472

I had chatGPT organize the next cell and add English translations to the terms.

In [ ]:
see_dr_pattern = (
    r"\b("
    # --- Common actions related to visiting or consulting a doctor ---
    r"مراجع[ةه]|"          # follow-up / review / check-up
    r"استشار[ةه]|"          # consultation
    r"زيار[ةه]|"            # visit
    r"راجع|"                 # see / visit (imperative)
    r"تراجع|"               # to follow up / you should see
    r"المتابع[هة](?:\s+مع)?" # follow-up with
    r")\s*"
    # --- Doctor entity ---
    r"(الطبيب|طبيبك|الدكتور|طبيب)"  # the doctor / your doctor
)


urgency_pattern = (
    r"(?:"
    # --- Basic words meaning "urgent" ---
    r"ضرور(?:ي|ي[هة])(?:\s*جداً| جدًا)?|"       # necessary / very important
    r"عاجل(?:[هة])?(?:اً)?|"                     # urgent
    # --- Immediate action terms ---
    r"فور(?:اً)?|"                               # immediately
    r"حال(?:اً)?|"                               # right away
    r"على\s+الفور|"                              # immediately / instantly
    r"سريع(?:اً)?|"                              # quickly
    # --- "As soon as possible" phrases ---
    r"(?:في|ب)\s+(?:أسرع|أقرب)\s+(?:وقت|فرص(?:ة|ه))(?:\s*ممكن(?:[هة])?)?|"  # as soon / as early as possible
    r"بأسرع\s+(?:ما\s+يمكن)|"                   # as fast as possible
    r"في\s+(?:أقرب|أسرع)\s+(?:فرص(?:ة|ه)\s*ممكن(?:[هة])?)?|"  # at the earliest opportunity
    # --- Delay avoidance ---
    r"دون\s+تأخير|بدون\s+تأخير|"                # without delay
    r"(?:يُرجى|يرجى|يفضل)\s+عدم\s+التأخير|"      # please / prefer not to delay
    # --- Structured urgency phrases ---
    r"بشكل\s+عاجل(?:[هة])?|بصورة\s+عاجل(?:ة|ه)|"  # urgently / in an urgent manner
    r"ضرورة\s+عاجل(?:ة|ه)|"                      # urgent necessity
    r"بأقصى\s+سرع(?:ة|ه)|"                       # with maximum speed
    # --- Soft urgency / importance phrasing ---
    r"من\s+(?:الضرور(?:ي|ي[هة])|المهم)\s+(?:مراجعة|زيارة)\s+الطبيب(?:\s+قريب(?:اً)?)?|"
    r"يفضل\s+في\s+(?:أقرب|أسرع)\s+(?:وقت|فرص(?:ة|ه))"
    r")"
)

def extract_keywords(pattern, text):
    return re.findall(pattern, text)

# Step 1: select rows that contain doctor pattern
df_see_dr = df[df['Answer'].str.contains(see_dr_pattern, regex=True, na=False)].copy()

# Step 2: extract doctor + urgency matches
df_see_dr["see_dr_matches"] = df_see_dr["Answer"].apply(lambda x: extract_keywords(see_dr_pattern, x))
df_see_dr["urgency_matches"] = df_see_dr["Answer"].apply(lambda x: extract_keywords(urgency_pattern, x))

# Step 3: merge all matches in one column (flatten)
df_see_dr["all_matches"] = df_see_dr.apply(
    lambda row: row["see_dr_matches"] + row["urgency_matches"], axis=1
)

# Step 4: add original index
df_see_dr["original_index"] = df_see_dr.index

# Step 5: mark urgency
df_see_dr["urgency_word"] = df_see_dr["urgency_matches"].apply(lambda x: len(x) > 0)

# Step 6: put urgency rows on top
df_final = df_see_dr.sort_values(by="urgency_word", ascending=False).reset_index(drop=True)
df_final.drop(columns=['see_dr_matches','urgency_matches'], inplace=True)
df_final


We can now work with the 'urgent' ones only..

In [6]:
df_final = df_final[df_final['urgency_word']== True].reset_index(drop=True)

In [7]:
df_final

,Question,Answer,Category,all_matches,original_index,urgency_word
0,عندي الم حاد في اعلى المعده عند اكل المأكولات ...,اخي الكريم يجب عليك مراجعة طبيب للباطنيه لتشخي...,أمراض الجهاز الهضمي,"[(مراجعة, طبيب), حال]",686950,True
1,استعملت قطرة لتوسيع حدقة العين وتوسعت، وآلان ل...,قطرة الميدرالين التي تستخدم لتوسيع الحدقة ذلك ...,امراض العيون,"[(راجع, طبيب), حال]",689317,True
2,أبلغ من العمر ٢٧ سنة تؤلمني رجلي عند ممارسة ال...,التشخيص يعتمد على مكان الالم هل هو في الركبة أ...,أمراض العضلات والعظام و المفاصل,"[(مراجعة, طبيب), حال]",689209,True
3,السلام عليكمانااسمي عبدالله .وقد قمت بعمل جراح...,عليك مراجعة طبيبك فورا,أمراض العضلات والعظام و المفاصل,"[(مراجعة, طبيبك), فور]",438318,True
4,اعاني من قيئ شديد وغثيان وانا في الشهر 3و استع...,يجب زياره طبيب نسائي اذ من الضروري اجراء الفحو...,أمراض الجهاز الهضمي,"[(زياره, طبيب), ضروري]",188614,True
...,...,...,...,...,...,...
17540,بشعر بغثيان ومغص وعندي كمان امساك ونفاخ واستفر...,لا يمكن التشخيص فورا الا باجراء تنظير علوي وسف...,أمراض الجهاز الهضمي,"[(مراجعة, طبيب), فور, حال]",76918,True
17541,عندي الم في الرجل اليسرى من الاعلى الى الاسفل ...,لابد من مراجعة طبيب لتقييم الحاله,أمراض العضلات والعظام و المفاصل,"[(مراجعة, طبيب), حال]",52292,True
17542,اني امارس العاده السريه بنفس الوقت العب كمال ا...,لابد من مراجعة طبيب لتقييم الحاله,أمراض العضلات والعظام و المفاصل,"[(مراجعة, طبيب), حال]",52237,True
17543,اعاني من الم في الساقين من الركبه الى الاسفل ح...,لابد من مراجعة طبيب لتقييم الحاله,أمراض العضلات والعظام و المفاصل,"[(مراجعة, طبيب), حال]",52298,True


In [8]:
calming_expressions = [
    "لا مشكلة",              # no problem
    "لا تقلق",               # don’t worry
    "اطمئن",                 # rest assured
    "الأمر طبيعي",           # it’s normal
    "طبيعي",                 # normal (general)
    "لا يحتاج علاج",         # no treatment needed
    "لا تحتاج دواء",         # no medicine needed
    "لا يحتاج تدخل",         # no intervention needed
    "مجرد إرهاق",            # just fatigue
    "مجرد قلق",              # just anxiety
    "غير خطير",              # not serious
    "كل شيء بخير",           # everything is fine
    "كل شي بخير",            # colloquial variant
    "لا داعي للقلق",         # no need to worry
    "لا داعي للخوف",         # no need to be afraid
    "لا يوجد ما يقلق",       # there’s nothing to worry about
    "حالة بسيطة",            # simple/mild case
    "حالة مؤقتة",            # temporary case
    "أمر شائع",              # common issue
    "شيء طبيعي",             # it’s something normal
    "نتائج طبيعية",          # normal results
    "يزول من تلقاء نفسه",    # goes away on its own
    "سيتحسن مع الوقت",       # will improve with time
    "مجرد عرض مؤقت",         # just a temporary symptom
    "لا يستدعي القلق",       # not a cause for concern
]


In [9]:
pattern = '|'.join([re.escape(k) for k in calming_expressions])
# Exclude statements with calming expressions
df_final = df_final[~df_final['Answer'].str.contains(pattern, regex=True, na=False)].reset_index(drop=True)

In [10]:
df_final

,Question,Answer,Category,all_matches,original_index,urgency_word
0,عندي الم حاد في اعلى المعده عند اكل المأكولات ...,اخي الكريم يجب عليك مراجعة طبيب للباطنيه لتشخي...,أمراض الجهاز الهضمي,"[(مراجعة, طبيب), حال]",686950,True
1,استعملت قطرة لتوسيع حدقة العين وتوسعت، وآلان ل...,قطرة الميدرالين التي تستخدم لتوسيع الحدقة ذلك ...,امراض العيون,"[(راجع, طبيب), حال]",689317,True
2,أبلغ من العمر ٢٧ سنة تؤلمني رجلي عند ممارسة ال...,التشخيص يعتمد على مكان الالم هل هو في الركبة أ...,أمراض العضلات والعظام و المفاصل,"[(مراجعة, طبيب), حال]",689209,True
3,السلام عليكمانااسمي عبدالله .وقد قمت بعمل جراح...,عليك مراجعة طبيبك فورا,أمراض العضلات والعظام و المفاصل,"[(مراجعة, طبيبك), فور]",438318,True
4,اعاني من قيئ شديد وغثيان وانا في الشهر 3و استع...,يجب زياره طبيب نسائي اذ من الضروري اجراء الفحو...,أمراض الجهاز الهضمي,"[(زياره, طبيب), ضروري]",188614,True
...,...,...,...,...,...,...
15029,بشعر بغثيان ومغص وعندي كمان امساك ونفاخ واستفر...,لا يمكن التشخيص فورا الا باجراء تنظير علوي وسف...,أمراض الجهاز الهضمي,"[(مراجعة, طبيب), فور, حال]",76918,True
15030,عندي الم في الرجل اليسرى من الاعلى الى الاسفل ...,لابد من مراجعة طبيب لتقييم الحاله,أمراض العضلات والعظام و المفاصل,"[(مراجعة, طبيب), حال]",52292,True
15031,اني امارس العاده السريه بنفس الوقت العب كمال ا...,لابد من مراجعة طبيب لتقييم الحاله,أمراض العضلات والعظام و المفاصل,"[(مراجعة, طبيب), حال]",52237,True
15032,اعاني من الم في الساقين من الركبه الى الاسفل ح...,لابد من مراجعة طبيب لتقييم الحاله,أمراض العضلات والعظام و المفاصل,"[(مراجعة, طبيب), حال]",52298,True


Now read the English one

In [11]:
df_eng = pd.read_excel("AHD_english.xlsx")

In [12]:
df_eng.head()

,Question,Answer,Category
0,"A breast MRI was performed, showing a non-mass...",What is important in the matter is what is the...,General Surgery
1,I suffer from things that force me to do thing...,"Peace be upon you, you need a psychotherapist ...",Psychiatric illness
2,I want to inquire about the complications of b...,"Like other surgical operations, the process of...",Endocrine diseases
3,Today I feel that I have something inside my a...,The condition is most likely to reflect third-...,General Surgery
4,Why do I suddenly feel like I don't know how t...,Possibly organic or a side effect of any medic...,Psychiatric illness


Now, we just get the corresponding English translations and concat them..

In [13]:
df_eng_selected = df_eng.loc[df_final['original_index']]

df_eng_selected = df_eng_selected.rename(columns=lambda c: f"{c}_eng")

df_eng_selected = df_eng_selected.reset_index(drop=True)
df_final = df_final.reset_index(drop=True)

df_final = pd.concat([df_final, df_eng_selected], axis=1)


In [14]:
df_final

,Question,Answer,Category,all_matches,original_index,urgency_word,Question_eng,Answer_eng,Category_eng
0,عندي الم حاد في اعلى المعده عند اكل المأكولات ...,اخي الكريم يجب عليك مراجعة طبيب للباطنيه لتشخي...,أمراض الجهاز الهضمي,"[(مراجعة, طبيب), حال]",686950,True,I have severe pain in the upper stomach when I...,"My dear brother, you must see an internal medi...",Gastrointestinal diseases
1,استعملت قطرة لتوسيع حدقة العين وتوسعت، وآلان ل...,قطرة الميدرالين التي تستخدم لتوسيع الحدقة ذلك ...,امراض العيون,"[(راجع, طبيب), حال]",689317,True,I used a drop to dilate the pupil and it dilat...,"Medralin drops, which are used to dilate the p...",eyes illnesses
2,أبلغ من العمر ٢٧ سنة تؤلمني رجلي عند ممارسة ال...,التشخيص يعتمد على مكان الالم هل هو في الركبة أ...,أمراض العضلات والعظام و المفاصل,"[(مراجعة, طبيب), حال]",689209,True,I am 27 years old. My legs hurt when I exercis...,The diagnosis depends on the location of the p...,Musculoskeletal and joint diseases
3,السلام عليكمانااسمي عبدالله .وقد قمت بعمل جراح...,عليك مراجعة طبيبك فورا,أمراض العضلات والعظام و المفاصل,"[(مراجعة, طبيبك), فور]",438318,True,Peace be upon you. My name is Abdullah. I had ...,You should see your doctor immediately,Musculoskeletal and joint diseases
4,اعاني من قيئ شديد وغثيان وانا في الشهر 3و استع...,يجب زياره طبيب نسائي اذ من الضروري اجراء الفحو...,أمراض الجهاز الهضمي,"[(زياره, طبيب), ضروري]",188614,True,"I suffer from severe vomiting and nausea, and ...","You must visit a gynecologist, as it is necess...",Gastrointestinal diseases
...,...,...,...,...,...,...,...,...,...
15029,بشعر بغثيان ومغص وعندي كمان امساك ونفاخ واستفر...,لا يمكن التشخيص فورا الا باجراء تنظير علوي وسف...,أمراض الجهاز الهضمي,"[(مراجعة, طبيب), فور, حال]",76918,True,"I feel nauseous and cramping, and I also have ...",Diagnosis cannot be made immediately except by...,Gastrointestinal diseases
15030,عندي الم في الرجل اليسرى من الاعلى الى الاسفل ...,لابد من مراجعة طبيب لتقييم الحاله,أمراض العضلات والعظام و المفاصل,"[(مراجعة, طبيب), حال]",52292,True,"I have pain in my left leg from top to bottom,...",You must see a doctor to evaluate the condition,Musculoskeletal and joint diseases
15031,اني امارس العاده السريه بنفس الوقت العب كمال ا...,لابد من مراجعة طبيب لتقييم الحاله,أمراض العضلات والعظام و المفاصل,"[(مراجعة, طبيب), حال]",52237,True,I practice masturbation at the same time as I ...,You must see a doctor to evaluate the condition,Musculoskeletal and joint diseases
15032,اعاني من الم في الساقين من الركبه الى الاسفل ح...,لابد من مراجعة طبيب لتقييم الحاله,أمراض العضلات والعظام و المفاصل,"[(مراجعة, طبيب), حال]",52298,True,I suffer from pain in the legs from the knee d...,You must see a doctor to evaluate the condition,Musculoskeletal and joint diseases


We will keep track of the last df, but it might be also useful to extract questions separately as these will be what we run the classification task over.

In [21]:
df_questions = df_final[['Question','Question_eng']]

In [22]:
df_final.to_csv("df_final.csv", index=False, encoding="utf-8-sig")
df_questions.to_csv("df_questions.csv", index=False, encoding="utf-8-sig")

************************************************

*******************************************

Now, we can consider having non-trigger data... a simple way can be to reverse the process...

In [23]:
negated_see_dr_pattern = (
    r"(?:"
        # --- Negation phrases (required) ---
        r"لا\s+داعي|"
        r"لا\s+تحتاج(?:\s+إلى)?|"
        r"لا\s+حاجة(?:\s+إلى|)\s*|"
        r"لست\s+بحاجة(?:\s+إلى)?|"
        r"لا\s+يوجد\s+ما\s+يستدعي|"
        r"لا\s+يستدعي\s+الأمر|"
        r"ليس\s+هناك\s+ما\s+يستدعي"
    r")\s*"
    r"(?:"
        # --- Actions (optional) ---
        r"مراجع[ةه]|"
        r"استشار[ةه]|"
        r"زيار[ةه]|"
        r"راجع|"
        r"تراجع|"
        r"المتابع[هة](?:\s+مع)?"
    r")?\s*"
    r"(?:"
        # --- Doctor entity (required) ---
        r"الطبيب|طبيبك|الدكتور|طبيب"
    r")"
)


calming_expressions = [ #Same as before, just repeating in case we wanted to work on this separately
    "لا مشكلة",              # no problem
    "لا تقلق",               # don’t worry
    "اطمئن",                 # rest assured
    "الأمر طبيعي",           # it’s normal
    "طبيعي",                 # normal (general)
    "لا يحتاج علاج",         # no treatment needed
    "لا تحتاج دواء",         # no medicine needed
    "لا يحتاج تدخل",         # no intervention needed
    "مجرد إرهاق",            # just fatigue
    "مجرد قلق",              # just anxiety
    "غير خطير",              # not serious
    "كل شيء بخير",           # everything is fine
    "كل شي بخير",            # colloquial variant
    "لا داعي للقلق",         # no need to worry
    "لا داعي للخوف",         # no need to be afraid
    "لا يوجد ما يقلق",       # there’s nothing to worry about
    "حالة بسيطة",            # simple/mild case
    "حالة مؤقتة",            # temporary case
    "أمر شائع",              # common issue
    "شيء طبيعي",             # it’s something normal
    "نتائج طبيعية",          # normal results
    "يزول من تلقاء نفسه",    # goes away on its own
    "سيتحسن مع الوقت",       # will improve with time
    "مجرد عرض مؤقت",         # just a temporary symptom
    "لا يستدعي القلق",       # not a cause for concern
]


In [ ]:
pattern = '|'.join([re.escape(k) for k in calming_expressions])
# Exclude statements with calming expressions
df[df['Answer'].str.contains(pattern, regex=True, na=False)].reset_index(drop=True)

,Question,Answer,Category
0,كيف تقضي المرضع صيام رمضان؟ فأنا أم مرضعة لطفل...,بداية أود التوضيح أنه يمكن للمرأة المرضع الصيا...,صحة الطفل
1,كيف تقضي الحامل صيام رمضان؟ فأنا حامل في الشهر...,بداية أود الإشارة إلى أنه من الجيد أنكِ قمتِ ب...,أمراض نسائية
2,كيف اتخلص من الاسهال في دقائق؟ لأنني أعاني من ...,أتمنى لك السلامة والتوفيق في امتحانك، وفي الحق...,أمراض الجهاز الهضمي
3,التحليل النسب فيه 27 اول ساعه وثاني ساعه 49 هل...,الطبيعي في هذا العم لغاية 20,الطب العام
4,ما هي الأعشاب المفيدة لضغط الدم المرتفع؟ فأنا ...,بداية أود التوضيح أنه من المهم استشارة الطبيب ...,ارتفاع ضغط الدم
...,...,...,...
85893,اعانى من الام مزمنة فى الركبة اليسرى و لا استط...,يبدو أنه قد حصل انسداد في أحد الشرينات الذي يغ...,أمراض العضلات والعظام و المفاصل
85894,ما هو الدواء المناسب لآلام الظهر من الأسفل؟,علاج وجع الظهر من الأسفل يعتمد على شدة الألم و...,أمراض العضلات والعظام و المفاصل
85895,طفله عمرها اربع سنوات تعاني من شعر خفيف جدا وق...,قلة كمية الشعر\nاسبابها\n1- خفة الشعر الطبيعيه...,البشرة والجمال
85896,أبي يبلغ من العمر 57 سنة ولديه في عينه اليمنى ...,للأسف لا يوجد علاج غير استئصال الظفرة الجراحي....,امراض العيون


Obviously here we cannot depend on calming expressions, because doctors usually use them even in serious cases

In [27]:
df[df['Answer'].str.contains(negated_see_dr_pattern, regex=True, na=False)].reset_index(drop=True)

,Question,Answer,Category
0,اشعر احيانا بالم مفاجئ في منطقة المستقيم وكان ...,تحتاج فحص من قبل اخصائي جراحة عامة أو جعاز هضم...,أمراض المسالك البولية والتناسلية
1,عندي القولون العصبي واعطاني استشاري الباطنية د...,لا تحتاج مراجعة طبيب نفسى اذا كان عندك توتر عص...,أمراض باطنية
2,هل سماع صوت احتكاك في الصدر ملحوظ وغير طبيعي ا...,لا تحتاج إلى زيارة الطبيب المختص العظام,صحة عامة


HAHAHAHA only 3; either white or black :D

In [28]:
pattern = '|'.join([re.escape(k) for k in calming_expressions])
# Exclude statements with calming expressions
df_calming = df[df['Answer'].str.contains(pattern, regex=True, na=False)].reset_index(drop=True)

Okay, we will take the calming ones and remove texts with the previously used see_dr and urgent patterns

In [ ]:
print('Occurences of see_dr pattern:  ' , len(df_calming[df_calming['Answer'].str.contains(see_dr_pattern)]))
print('Occurences of urgency pattern:  ' , len(df_calming[df_calming['Answer'].str.contains(urgency_pattern)]))

In [ ]:
df_calming = df_calming[~df_calming['Answer'].str.contains(see_dr_pattern)]
df_calming = df_calming[~df_calming['Answer'].str.contains(urgency_pattern)]

In [33]:
df_calming

,Question,Answer,Category
2,كيف اتخلص من الاسهال في دقائق؟ لأنني أعاني من ...,أتمنى لك السلامة والتوفيق في امتحانك، وفي الحق...,أمراض الجهاز الهضمي
3,التحليل النسب فيه 27 اول ساعه وثاني ساعه 49 هل...,الطبيعي في هذا العم لغاية 20,الطب العام
7,اشعر ب مرض نفسي بسبب حادث في الطفوله تعرض للاغ...,الف سلامه عليكي انت تحتاجين لمعالج نفسي لمساعد...,أمراض نفسية
8,السلام عليكم انا كان لدي مشاكل نفسيه شديده في ...,نعم طبيعي لانه يذكرك بموقف صعب مررتي بيه فاصبح...,أمراض نفسية
9,ما هو معدل الضغط الدموي بالنسبة لسن 80\nوكذالك...,علي حسب جمعيه القلب الأمريكيةفإن معدل ضغط الدم...,الطب العام
...,...,...,...
85888,انا مصاب بداء نقص تروية راس الفخذ وعمري هو ٣٢ ...,السبب الرئيس هو نقص التروية الدموية في رسأس ال...,أمراض العضلات والعظام و المفاصل
85891,بنتي عهرها 10 سنوات واصيبت بالعصب السابع وترك ...,يصعب علاج شلل العصب السابع كلما تأخر العلاج، و...,علم التشريح
85892,اعاني منذ فترة خمس سنوات من اصابة العصب السابع...,يصعب علاج شلل العصب السابع كلما تأخر العلاج، و...,علم التشريح
85893,اعانى من الام مزمنة فى الركبة اليسرى و لا استط...,يبدو أنه قد حصل انسداد في أحد الشرينات الذي يغ...,أمراض العضلات والعظام و المفاصل
